## Name : Md Rasel
### Matriculation no: 2130710

In [ ]:
import jupedsim as jps
import pedpy
from pedpy import WalkableArea, MeasurementArea
import shapely 
from shapely import Polygon, Point
import pathlib
import numpy as np

area = Polygon([(0, 0), (20, 0), (20, 20), (0, 20)])
walkable_area = WalkableArea(area)
measurement_area = MeasurementArea(area)

spawning_area = Polygon([(2, 2), (18, 2), (18, 18), (2, 18)])
num_agents_per_exit = 25
num_exits = 4
total_agents = num_agents_per_exit * num_exits

exit_areas = [
    [(4, 0), (5, 0), (5, 0.2), (4, 0.2)],
    [(15, 0), (16, 0), (16, 0.2), (15, 0.2)],
    [(4, 19.8), (4, 20), (5, 20), (5, 19.8)],
    [(15, 19.8), (15, 20), (16, 20), (16, 19.8)],
]

pos_in_spawning_area = jps.distributions.distribute_by_density(
    polygon=spawning_area,
    density=total_agents / spawning_area.area,
    distance_to_agents=0.5,
    distance_to_polygon=0.7,
    seed=45131502,
)

trajectory_file = "Test9.sqlite"  # output file
simulation = jps.Simulation(
    model=jps.CollisionFreeSpeedModel(),
    geometry=area,
    trajectory_writer=jps.SqliteTrajectoryWriter(
        output_file=pathlib.Path(trajectory_file)
    ),
)

exit_ids = []
journey_ids = []

for exit_area in exit_areas:
    exit_id = simulation.add_exit_stage(exit_area)
    exit_ids.append(exit_id)
    
    journey = jps.JourneyDescription([exit_id])
    journey_id = simulation.add_journey(journey)
    journey_ids.append(journey_id)

ids = []

v_distribution = np.random.normal(1.2, 0.05, total_agents)

for pos, v0 in zip(pos_in_spawning_area, v_distribution):
    min_distance = float('inf')
    selected_journey_id = None
    selected_exit_id = None
    
    for i, exit_area in enumerate(exit_areas):
        distance = Point(pos).distance(Polygon(exit_area))
        if distance < min_distance:
            min_distance = distance
            selected_journey_id = journey_ids[i]
            selected_exit_id = exit_ids[i]
    
    agent_params = jps.CollisionFreeSpeedModelAgentParameters(
        journey_id=selected_journey_id,
        stage_id=selected_exit_id,  
        position=pos,
        v0=v0,
        radius=0.15,
    )
    id = simulation.add_agent(agent_params)
    ids.append(id)

while simulation.agent_count() > 0:
    simulation.iterate()
      


In [ ]:
from jupedsim.internal.notebook_utils import animate, read_sqlite_file

trajectory_data, walkable_area = read_sqlite_file(trajectory_file)
animate(trajectory_data, walkable_area, every_nth_frame=10)

# Analysis